In [1]:
import sys
import pickle
import torch
import os
import numpy as np
import mxnet as mx
from mxnet import ndarray as nd

sys.path.append("/app/sandbox/AdaFace")
from AdaFace.data import test_dataset, prepare_data
from AdaFace.evaluate_utils import get_val_data

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [2]:
def load_bin(path, image_size=[112, 112]):
    try:
        with open(path, "rb") as f:
            bins, issame_list = pickle.load(f)  # py2
    except UnicodeDecodeError as e:
        with open(path, "rb") as f:
            bins, issame_list = pickle.load(f, encoding="bytes")  # py3
    data_list = []
    for flip in [0, 1]:
        data = torch.empty((len(issame_list) * 2, 3, image_size[0], image_size[1]))
        data_list.append(data)
    for idx in range(len(issame_list) * 2):
        _bin = bins[idx]
        img = mx.image.imdecode(_bin)
        if img.shape[1] != image_size[0]:
            img = mx.image.resize_short(img, image_size[0])
        img = nd.transpose(img, axes=(2, 0, 1))
        for flip in [0, 1]:
            if flip == 1:
                img = mx.ndarray.flip(data=img, axis=2)
            data_list[flip][idx][:] = torch.from_numpy(img.asnumpy())
        if idx % 1000 == 0:
            print("loading bin", idx)
    print(data_list[0].shape)
    return data_list, issame_list

In [4]:
# data_set = load_bin('/app/datasets/ms1m/agedb_30.bin')
# len(data_set[1]), np.sum(data_set[1])
# data_set[0][0].shape

In [2]:
data_root = "/app/datasets"
val_data_path = "ms1m"
val_data = get_val_data(os.path.join(data_root, val_data_path))
# theses datasets are already normalized with mean 0.5, std 0.5
(
    age_30,
    cfp_fp,
    lfw,
    age_30_issame,
    cfp_fp_issame,
    lfw_issame,
    cplfw,
    cplfw_issame,
    calfw,
    calfw_issame,
) = val_data
val_data_dict = {
    "agedb_30": (age_30, age_30_issame),
    "cfp_fp": (cfp_fp, cfp_fp_issame),
    "lfw": (lfw, lfw_issame),
    "cplfw": (cplfw, cplfw_issame),
    "calfw": (calfw, calfw_issame),
}

laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile


In [8]:
age_30.shape

(12000, 3, 112, 112)

In [25]:
val_data_dict["agedb_30"][0].shape

(12000, 3, 112, 112)

In [23]:
len(val_data_dict["agedb_30"][1]), np.sum(val_data_dict["agedb_30"][1])

(6000, 3000)

In [29]:
np.all(np.array(data_set[1]) == val_data_dict["agedb_30"][1])

True

In [3]:
prepare_data(data_root, val_data_path, "five_mem_concat")

In [4]:
five_ds = test_dataset(data_root, val_data_path, "five_mem_concat")

laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile
laoding validation data memfile


In [5]:
len(five_ds)

62000

In [9]:
len(five_ds[0][])

4

In [8]:
five_ds[0][0].shape

torch.Size([3, 112, 112])